En este archivo, voy a desarrollar la función que se me pide en el cuarto endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_4.png "Title")

# Importaciones necesarias para trabajar

In [1]:
import pandas as pd

Archivos que voy a usar para desarrollar este endpoit

In [6]:
reseñas = pd.read_parquet("../EDA/user_reviews_complete.parquet")
reseñas.head(2)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479


In [5]:
juegos = pd.read_parquet("../Datasets/steam_games_complete.parquet")
juegos.head(2)

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00
